#Revised Simplex Method

In [ ]:
import numpy as np
import sys
import time

In [ ]:
class LinearModel:
    
    def __init__(self, A = np.empty([0,0]), b = np.empty([0,0]), c = np.empty([0,0]), minmax = "MAX"):
        self.A = A
        self.b = b
        self.c = c
        self.x = [float(0)] * len(c)
        self.minmax = minmax
        self.optimalValue = None
        self.transform = False
        self.OptimizedValue=0
        self.table1 = 0
        self.table2 = 0
        
    def addA(self, A):
        self.A = A
        
    def addB(self, b):
        self.b = b
        
    def addC(self, c):
        self.c = c
        self.transform = False
    
    def setObj(self, minmax):
        if(minmax == "MIN" or minmax == "MAX"):
            self.minmax = minmax
        else:
            print("Invalid objective.")
        self.transform = False
    
    def printSoln(self):
        print(self.OptimizedValue)
            
    def getTableau(self):
        # construct starting tableau
        
        if(self.minmax == "MIN" and self.transform == False):
            self.c[0:len(c)] = -1 * self.c[0:len(c)]
            self.transform = True
        
        numVar = len(self.c)
        numSlack = len(self.A)

        self.table1 = np.zeros([1+numSlack,4+numSlack])
        self.table2=np.zeros([1+numSlack,numVar])
        
        basis = np.array([0] * numSlack)
        for i in range(0, len(basis)):
            basis[i] = numVar + i
        
        self.table1[:, 0:(1+numSlack)] = np.identity(numSlack+1)
        self.table1[0,numSlack+1]=0
        self.table1[1:,numSlack+1]=np.transpose(self.b)
        self.table1 = np.array(self.table1, dtype ='float')

        self.table2[0,:]=self.c
        self.table2[1:,:]=self.A
        self.table2 = np.array(self.table2, dtype ='float')
            
    def optimize(self):
        
        numVar = len(self.c)
        numSlack = len(self.A)

        if(self.minmax == "MIN" and self.transform == False):
            for i in range(numVar):
                self.c[i] = -1 * self.c[i]
                transform = True
        
        self.getTableau()

        # keep track of iterations for display
        iter = 1

        while(1):

            print('------------------------------------------')
            print('Iteration: ', iter)
            print(self.table1)

            min_delta=9999

            optimal = True
            
            pivot_i0=0

            for i in range(numVar):
              delta=np.matmul(self.table1[0,0:(1+numSlack)],self.table2[:,i])

              if delta<0:
                optimal=False

                if delta<min_delta:
                  min_delta=delta
                  pivot_i0=i

            # if all directions result in decreased profit or increased cost
            if optimal == True: 
                 break
            
            new_vector = self.table2[:,pivot_i0]
            self.table1[:,numSlack+2]=np.matmul(self.table1[:,0:(1+numSlack)],new_vector)

            pivot_i1=0
            min_val=9999

            for i in range(1+numSlack):

              if self.table1[i,numSlack+2]>0:
                self.table1[i,numSlack+3]=self.table1[i,numSlack+1]/self.table1[i,numSlack+2]

                if self.table1[i,numSlack+3]<min_val:
                  min_val=self.table1[i,numSlack+3]
                  pivot_i1=i

            key_element=self.table1[pivot_i1,numSlack+2]
            
            self.table2[:,pivot_i0]=self.table1[:,pivot_i1]

            for i in range(1,numSlack+2):
              self.table1[pivot_i1,i]=self.table1[pivot_i1,i]/key_element
            
            for i in range(1+numSlack):
              if i!=pivot_i1:
                self.table1[i,1:numSlack+2]=self.table1[i,1:numSlack+2]-self.table1[i,numSlack+2]*self.table1[pivot_i1,1:numSlack+2]
            
            iter = iter + 1
        
        self.OptimizedValue = self.table1[0,numSlack+1]

In [ ]:
model1 = LinearModel()

A = np.array([[2, 3], 
              [-3, 2], 
              [0, 2],
              [2, 1]])
b = np.array([6, 3, 5, 4])
#according to our convention we must flip the sign of coefficients of objective function
c = np.array([-4, -3])

model1.addA(A)
model1.addB(b)
model1.addC(c)
model1.setObj("MAX")

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n")

start = time.time()
model1.optimize()
end = time.time()

print("\nOptimized value:  ")
model1.printSoln()

time_revised_simplex = end-start

A =
 [[ 2  3]
 [-3  2]
 [ 0  2]
 [ 2  1]] 

b =
 [6 3 5 4] 

c =
 [-4 -3] 

------------------------------------------
Iteration:  1
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 6. 0. 0.]
 [0. 0. 1. 0. 0. 3. 0. 0.]
 [0. 0. 0. 1. 0. 5. 0. 0.]
 [0. 0. 0. 0. 1. 4. 0. 0.]]
------------------------------------------
Iteration:  2
[[ 1.   0.   0.   0.   2.   8.  -4.   0. ]
 [ 0.   1.   0.   0.  -1.   2.   2.   3. ]
 [ 0.   0.   1.   0.   1.5  9.  -3.   0. ]
 [ 0.   0.   0.   1.   0.   5.   0.   0. ]
 [ 0.   0.   0.   0.   0.5  2.   2.   2. ]]
------------------------------------------
Iteration:  3
[[ 1.          0.5         0.          0.          1.5         9.
  -1.          0.        ]
 [ 0.          0.5         0.          0.         -0.5         1.
   2.          1.        ]
 [ 0.         -1.75        1.          0.          3.25        5.5
   3.5         2.57142857]
 [ 0.         -1.          0.          1.          1.          3.
   2.          2.5       ]
 [ 0.         -0.25        0

#Simplex Method

In [ ]:
class LinearModel:
    
    def __init__(self, A = np.empty([0,0]), b = np.empty([0,0]), c = np.empty([0,0]), minmax = "MAX"):
        self.A = A
        self.b = b
        self.c = c
        self.x = [float(0)] * len(c)
        self.minmax = minmax
        self.printIter = True
        self.optimalValue = None
        self.transform = False
        
    def addA(self, A):
        self.A = A
        
    def addB(self, b):
        self.b = b
        
    def addC(self, c):
        self.c = c
        self.transform = False
    
    def setObj(self, minmax):
        if(minmax == "MIN" or minmax == "MAX"):
            self.minmax = minmax
        else:
            print("Invalid objective.")
        self.transform = False
            
    def setPrintIter(self, printIter):
        self.printIter = printIter
            
    def printSoln(self):
        print("Coefficients: ")
        print(self.x)
        print("Optimal value: ")
        print(self.optimalValue)
        
    def printTableau(self, tableau):
        
        print("ind \t\t", end = "")
        for j in range(0, len(c)):
            print("x_" + str(j), end = "\t")
        for j in range(0, (len(tableau[0]) - len(c) - 2)):
            print("s_" + str(j), end = "\t")
        
        print()
        for j in range(0, len(tableau)):
            for i in range(0, len(tableau[0])):
                if(not np.isnan(tableau[j, i])):
                    if(i == 0):
                        print(int(tableau[j, i]), end = "\t")
                    else:
                        print(round(tableau[j, i], 2), end = "\t")
                else:
                    print(end = "\t")
            print()
            
    def getTableau(self):
        # construct starting tableau
        
        if(self.minmax == "MIN" and self.transform == False):
            self.c[0:len(c)] = -1 * self.c[0:len(c)]
            self.transform = True
        
        t1 = np.array([None, 0])
        numVar = len(self.c)
        numSlack = len(self.A)
        
        t1 = np.hstack(([None], [0], self.c, [0] * numSlack))
        
        basis = np.array([0] * numSlack)
        
        for i in range(0, len(basis)):
            basis[i] = numVar + i
        
        A = self.A
        
        if(not ((numSlack + numVar) == len(self.A[0]))):
            B = np.identity(numSlack)
            A = np.hstack((self.A, B))
            
        t2 = np.hstack((np.transpose([basis]), np.transpose([self.b]), A))
        
        tableau = np.vstack((t1, t2))
        
        tableau = np.array(tableau, dtype ='float')
        
        return tableau
            
    def optimize(self):
        
        if(self.minmax == "MIN" and self.transform == False):
            for i in range(len(self.c)):
                self.c[i] = -1 * self.c[i]
                transform = True
        
        tableau = self.getTableau()
         
        if(self.printIter == True):
            print("Starting Tableau:")
            self.printTableau(tableau)
        
        # assume initial basis is not optimal
        optimal = False

        # keep track of iterations for display
        iter = 1

        while(True):
            
            if(self.printIter == True):
                print("----------------------------------")
                print("Iteration :", iter)
                self.printTableau(tableau)
                
            if(self.minmax == "MAX"):
                for profit in tableau[0, 2:]:
                    if profit > 0:
                        optimal = False
                        break
                    optimal = True
            else:
                for cost in tableau[0, 2:]:
                    if cost < 0:
                        optimal = False
                        break
                    optimal = True

            # if all directions result in decreased profit or increased cost
            if optimal == True: 
                 break
            
            # nth variable enters basis, account for tableau indexing
            if (self.minmax == "MAX"):
                n = tableau[0, 2:].tolist().index(np.amax(tableau[0, 2:])) + 2
            else:
                n = tableau[0, 2:].tolist().index(np.amin(tableau[0, 2:])) + 2

            # minimum ratio test, rth variable leaves basis 
            minimum = 99999
            r = -1

            for i in range(1, len(tableau)): 
                if(tableau[i, n] > 0):
                    val = tableau[i, 1]/tableau[i, n]
                    if val<minimum: 
                        minimum = val 
                        r = i
                            
            pivot = tableau[r, n] 
            
            print("Pivot Column:", n)
            print("Pivot Row:", r)
            print("Pivot Element: ", pivot)

            # perform row operations 
            # divide the pivot row with the pivot element 
            tableau[r, 1:] = tableau[r, 1:] / pivot 
            
            

            # pivot other rows
            for i in range(0, len(tableau)): 
                if i != r:
                    mult = tableau[i, n] / tableau[r, n]
                    tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:] 


            # new basic variable 
            tableau[r, 0] = n - 2
            
            iter += 1
            
        
        if(self.printIter == True):
            print("----------------------------------")
            print("Final Tableau reached in", iter, "iterations")
            self.printTableau(tableau)
        else:
            print("Solved")
            
        self.x = np.array([0] * len(c), dtype = float)
        # save coefficients
        for key in range(1, (len(tableau))):
            if(tableau[key, 0] < len(c)):
                self.x[int(tableau[key, 0])] = tableau[key, 1]
        
        self.optimalValue = -1 * tableau[0,1]

In [ ]:
model1 = LinearModel()

A = np.array([[2, 3], 
              [-3, 2], 
              [0, 2],
              [2, 1]])
b = np.array([6, 3, 5, 4])
c = np.array([4, 3])

model1.addA(A)
model1.addB(b)
model1.addC(c)
model1.setObj("MAX")

print("A =\n", A, "\n")
print("b =\n", b, "\n")
print("c =\n", c, "\n\n")

start = time.time()
model1.optimize()
end = time.time()

print("\n")
model1.printSoln()

time_simplex = end - start

A =
 [[ 2  3]
 [-3  2]
 [ 0  2]
 [ 2  1]] 

b =
 [6 3 5 4] 

c =
 [4 3] 


Starting Tableau:
ind 		x_0	x_1	s_0	s_1	s_2	s_3	
	0.0	4.0	3.0	0.0	0.0	0.0	0.0	
2	6.0	2.0	3.0	1.0	0.0	0.0	0.0	
3	3.0	-3.0	2.0	0.0	1.0	0.0	0.0	
4	5.0	0.0	2.0	0.0	0.0	1.0	0.0	
5	4.0	2.0	1.0	0.0	0.0	0.0	1.0	
----------------------------------
Iteration : 1
ind 		x_0	x_1	s_0	s_1	s_2	s_3	
	0.0	4.0	3.0	0.0	0.0	0.0	0.0	
2	6.0	2.0	3.0	1.0	0.0	0.0	0.0	
3	3.0	-3.0	2.0	0.0	1.0	0.0	0.0	
4	5.0	0.0	2.0	0.0	0.0	1.0	0.0	
5	4.0	2.0	1.0	0.0	0.0	0.0	1.0	
Pivot Column: 2
Pivot Row: 4
Pivot Element:  2.0
----------------------------------
Iteration : 2
ind 		x_0	x_1	s_0	s_1	s_2	s_3	
	-8.0	0.0	1.0	0.0	0.0	0.0	-2.0	
2	2.0	0.0	2.0	1.0	0.0	0.0	-1.0	
3	9.0	0.0	3.5	0.0	1.0	0.0	1.5	
4	5.0	0.0	2.0	0.0	0.0	1.0	0.0	
0	2.0	1.0	0.5	0.0	0.0	0.0	0.5	
Pivot Column: 3
Pivot Row: 1
Pivot Element:  2.0
----------------------------------
Iteration : 3
ind 		x_0	x_1	s_0	s_1	s_2	s_3	
	-9.0	0.0	0.0	-0.5	0.0	0.0	-1.5	
1	1.0	0.0	1.0	0.5	0.0	0.0	-0.5	
3	5.5	

Time comparisions:

In [ ]:
print('Time taken by simplex algorithm: ', time_simplex*1000, 'ms')
print('Time taken by revised simplex algorithm', time_revised_simplex*1000, 'ms')

Time taken by simplex algorithm:  14.704227447509766 ms
Time taken by revised simplex algorithm 2.381563186645508 ms
